# Part 1 - Scraping

In [1]:
#Dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import time
import requests
import pprint 
from IPython.display import Markdown, display
import pymongo
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/Sally/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


### NASA Mars News
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [4]:
# connect to NASA Mars news Site
url = 'https://redplanetscience.com/'

# Retrieve page with the requests module
browser.visit(url)

In [5]:
response = requests.get(url)
response

<Response [200]>

In [6]:
html=browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

# Examine the results, determine elements that contains sought info.
print(soup.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
  <title>
   News - Mars Exploration Program
  </title>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <span class="logo">
        MA

In [7]:
article = soup.find("div", class_ = "list_text")
news_title = article.find("div", class_="content_title").text
news_p = article.find("div", class_="article_teaser_body").text

print(f'------------------------------------------------')
print(f'TITLE: {news_title}')
print(f'------------------------------------------------')
print(f'PARAGRAPH: {news_p}')

------------------------------------------------
TITLE: NASA Updates Mars 2020 Mission Environmental Review
------------------------------------------------
PARAGRAPH: NASA and the Department of Energy have completed a more detailed risk analysis for the Mars 2020 rover launch from Florida.


### JPL Mars Space Images - Featured Image

In [8]:
# Visit the url for JPL Featured Space Image
# Set URL
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [9]:
html=browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [10]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
browser.find_by_css("a.showimg").first.click()
time.sleep(2)

In [11]:
#parse html page with BeautifulSoup
html=browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [12]:
# Need more info to find image url
image_url = browser.find_by_css("img.fancybox-image")["src"]
image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

### Mars Facts

In [13]:
# Visit the Mars Facts webpage
# Set URL
url = 'https://galaxyfacts-mars.com/'
browser.visit(url)

In [14]:
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_facts_df = pd.read_html("https://space-facts.com/mars/")[0]
print(mars_facts_df)

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


In [15]:
# Clean up DataFrame, set index
mars_facts_df.columns=["Planet Profile", "Value"]
mars_facts_df.set_index("Planet Profile", inplace=True)
mars_facts_df

,Value
Planet Profile,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [16]:
# Visit the Astrogeology site
# Set URL
url = 'https://marshemispheres.com/'
browser.visit(url)

In [17]:
# Parse Results HTML with BeautifulSoup
html = browser.html
mars_weather_soup = BeautifulSoup(html, "html.parser")

In [18]:
# Save both the image url string for the full resolution hemisphere image, 
# and the Hemisphere title containing the hemisphere name


soup1 = BeautifulSoup(html, "html.parser")
items = soup1.find_all("div", class_="item")

hemisphere_img_urls = []

for item in items:
    
    title = item.find("h3").text
    link = item.find("a", class_="itemLink")["href"]
    hemispherelink = url + link
    browser.visit(hemispherelink)
    hemispherehtml = browser.html
    
    soup2 = BeautifulSoup(hemispherehtml, "lxml")
    image = soup2.find("img", class_="wide-image")["src"]
    imageurl = url + image
    hemisphere = {}

    hemisphere_img_urls.append({"title":title,"img_url":imageurl})

    browser.back()


In [19]:
# Use a Python dictionary to store the data using the keys `img_url` and `title`
hemisphere_img_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]